In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

In [131]:
import scipy as sp

rv = sp.stats.norm()
type(rv)

scipy.stats._distn_infrastructure.rv_frozen

In [2]:
%matplotlib inline

In [3]:
prop = pd.read_csv("properties_2016.csv")
train = pd.read_csv("train_2016_v2.csv")
df_train = train.merge(prop, how="left", on = 'parcelid')
print(len(df_train))

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


90275


# pool

##### 각 변수의 null 이 의미
* poolcnt와 pooltypeid7 (Pool without hot tub)가 비어있지 않은 데이터를 약 20% 가지고 있다. 즉 2016년에 거래된 집의 20%는 pool을 가지고 있으며 그 중 대부분은 온탕(?)이 딸려있지 않은 pool이다.
    * poolcnt : pool없음 ('0'), 혹은 누락
    * poolsizesum: 누락, 혹은 pool없음('0')
    * poolpetypeid2,7,10: 해당 풀타입이 아님('0'), 혹은 누락
    * hashottuborspa : hot tub이나 spa가 없음 ('False'), 혹은 누락 
    
#####  null값 처리 계획
    * poolcnt : 0
    * poolsizesum : ?
    * pooltypeid2, pooltypeid7 : 0
    * pooltypeid10 : 0
    

In [8]:
prop_pools = ['poolcnt', 'poolsizesum','pooltypeid10', 'pooltypeid2', 'pooltypeid7','hashottuborspa']
traincolumns = ['parcelid', 'logerror']

In [19]:
df_pools = df_train[prop_pools+traincolumns]
df_pools.tail()

,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,hashottuborspa,parcelid,logerror
90270,1.0,NaN,NaN,NaN,1.0,NaN,10774160,-0.0356
90271,NaN,NaN,NaN,NaN,NaN,NaN,12046695,0.0070
90272,NaN,NaN,NaN,NaN,NaN,NaN,12995401,-0.2679
90273,NaN,NaN,NaN,NaN,NaN,NaN,11402105,0.0602
90274,NaN,NaN,NaN,NaN,NaN,NaN,12566293,0.4207


In [10]:
counts_pools = df_pools.describe().T[['count']].iloc[:5]
counts_pools.loc['hashottuborspa'] = df_pools.hashottuborspa.count()
counts_pools['null counts'] = counts_pools['count'].apply(lambda x: 90275 - x )
counts_pools["ratio"] = counts_pools["count"].apply(lambda x: x/90275).round(4)
counts_pools

,count,null counts,ratio
poolcnt,17901.0,72374.0,0.1983
poolsizesum,969.0,89306.0,0.0107
pooltypeid10,1161.0,89114.0,0.0129
pooltypeid2,1204.0,89071.0,0.0133
pooltypeid7,16697.0,73578.0,0.1850
hashottuborspa,2365.0,87910.0,0.0262


In [108]:
df_pools["Count"] = 1
pivot_pools = df_pools.Count.groupby([df_pools.poolcnt, df_pools.pooltypeid10,df_pools.pooltypeid2,df_pools.pooltypeid7]).sum().unstack('poolcnt')
pd.DataFrame(pivot_pools) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


poolcnt                                   0.0      1.0
pooltypeid10 pooltypeid2 pooltypeid7                  
0.0          0.0         0.0          71213.0      NaN
                         1.0              NaN  16697.0
             1.0         0.0              NaN   1204.0
1.0          0.0         0.0           1161.0      NaN

In [44]:
df_pools.Count.groupby([df_pools.poolcnt, df_pools.pooltypeid10, df_pools.hashottuborspa]).sum().unstack('poolcnt')

poolcnt                          0.0      1.0
pooltypeid10 hashottuborspa                  
0.0          False           71213.0  16697.0
             True                NaN   1204.0
1.0          True             1161.0      NaN

In [20]:
df_pools['hashottuborspa'].fillna(value=False,inplace=True)
df_pools.fillna(value=0.0, inplace=True)
df_pools.tail()

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:4355: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3035: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,hashottuborspa,parcelid,logerror
90270,1.0,0.0,0.0,0.0,1.0,False,10774160,-0.0356
90271,0.0,0.0,0.0,0.0,0.0,False,12046695,0.0070
90272,0.0,0.0,0.0,0.0,0.0,False,12995401,-0.2679
90273,0.0,0.0,0.0,0.0,0.0,False,11402105,0.0602
90274,0.0,0.0,0.0,0.0,0.0,False,12566293,0.4207


### pooltypeid를 하나의 column으로 통합한다면 

In [143]:
df_pools['pooltypeid'] = df_pools.pooltypeid10*10 + df_pools.pooltypeid2*2 + df_pools.pooltypeid7*7
df_pools.tail()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,poolcnt,poolsizesum,pooltypeid10,pooltypeid2,pooltypeid7,hashottuborspa,parcelid,logerror,pooltypeid
90270,1.0,0.0,0.0,0.0,1.0,False,10774160,-0.0356,7.0
90271,0.0,0.0,0.0,0.0,0.0,False,12046695,0.0070,0.0
90272,0.0,0.0,0.0,0.0,0.0,False,12995401,-0.2679,0.0
90273,0.0,0.0,0.0,0.0,0.0,False,11402105,0.0602,0.0
90274,0.0,0.0,0.0,0.0,0.0,False,12566293,0.4207,0.0
